In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
sns.reset_orig()

from matplotlib import rc
rc('text', usetex=True)
rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{siunitx}', r'\usepackage{amsmath}']}
plt.rcParams.update(params)

from scipy.stats import bernoulli
from scipy.special import loggamma

from sklearn.preprocessing import OneHotEncoder

import tqdm

from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

In [ ]:
import os
import pickle

In [ ]:
from joblib import Parallel, delayed

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', UserWarning)

In [ ]:
EPS = np.finfo(np.float).eps

In [ ]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [ ]:
indir = './data'
outdir = './output'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
# R packages
## blockmodels
blockmodels = importr("blockmodels")
## label.switching
label_switching = importr("label.switching")
## base
base = importr("base")

dollar = base.__dict__["$"]

In [ ]:
from functools import lru_cache

@lru_cache(maxsize = 10000)
def normterm_discrete(n, k):
    if n == 1:
        return np.log(k)
    if k == 1:
        return 1.0
    elif k == 2:
        return np.sum(sorted([ np.exp(loggamma(n+1) - loggamma(t+1) - loggamma(n-t+1) + 
                               t*(np.log(t) - np.log(n)) + (n-t)*(np.log(n-t) - np.log(n))
                        )
                        for t in range(1, n)]))
    else:
        return normterm_discrete(n, k-1) + n/(k-2) * normterm_discrete(n, k-2)

In [10]:
def check_latent_index_variable(z):
    unique_z = sorted(np.unique(z))
    if len(unique_z) == np.max(z) + 1:
        return z
    new_z = np.zeros(z.shape, dtype=np.int)
    for index, current in enumerate(unique_z):
        new_z[z == current] = index
    return new_z

In [11]:
def calc_dnml(X, Z1, Z2, K=3, L=3):    
    N = X.shape[0]
    
    codelen_x_z = 0.0
    codelen_z = 0.0

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen_x_z += n_all * np.log(n_all)
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
            
            if n_pos >=2:
                codelen_x_z -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen_x_z -= n_neg * np.log(n_neg)
            
        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen_z += n_k * (np.log(N) - np.log(n_k))

    codelen_z += np.log(normterm_discrete(N, K))
    
    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [12]:
def calc_dnml_by_prob(X, Z, alpha, theta, K=3, L=3, eps=1e-12):    
    N = X.shape[0]
    
    Z1 = np.argmax(Z, axis=1)
    
    Z1 = check_latent_index_variable(Z1)
    Z2 = Z1
    
    codelen_x_z = 0.0
    codelen_z = 0.0
        
    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg
            
            if theta[k, l] < eps:
                theta[k, l] = eps
            if theta[k, l] > 1.0 - eps:
                theta[k, l] = 1.0 - eps
            
            codelen_x_z += -n_pos * np.ma.log(theta[k, l]) - n_neg * np.ma.log(1.0 - theta[k, l])

            if n_all >=2:
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
    
        n_k = np.sum(Z1 == k)
        codelen_z += -n_k * np.log(alpha[k])

    codelen_z += np.log(normterm_discrete(N, K))

    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [13]:
def calc_stats(X, #z, 
               scores, scores_f, scores_l, h, delta, K=10):
    scores = np.array(scores)
    scores_f = np.array(scores_f)
    scores_l = np.array(scores_l)
    
    K = scores.shape[2]
    N_trial = scores.shape[0]
    T = scores.shape[1]
    
    codelens = np.array([ codelen_integer(k) for k in range(1, K+1)])
    
    idxes_all = np.argmin(scores + np.tile(codelens[np.newaxis, np.newaxis], (N_trial, T, 1))[0, 0, :], axis=2)
    
    models_estimated = np.nan * np.ones((N_trial, T), dtype=np.float)
    models_former = np.nan * np.ones((N_trial, T), dtype=np.float)
    models_latter = np.nan * np.ones((N_trial, T), dtype=np.float)
    stats_complete = np.nan * np.ones((N_trial, T), dtype=np.float)
    
    for trial in range(scores.shape[0]):
        n_change = 0  # number of changes so far.
        for t in range(h, T-h):
            alpha= (n_change+1/2) / (t+1+1)
            m_estimated = idxes_all[trial, t]

            # Lv.3 change (Model change)
            stats_half_t = np.zeros((K, K), dtype=np.float)
            for k1 in range(K):
                stats_former = scores_f[trial, t, k1]
                for k2 in range(K):
                    if k1 == k2:
                        p = 1.0 - alpha
                    else:
                        p = alpha/(K-1)
                    stats_latter = scores_l[trial, t, k2]
                    stats_half_t[k1, k2] = (stats_former + stats_latter) + codelens[k1] - np.log(p)
            m_former_estimated, m_latter_estimated = np.unravel_index(np.nanargmin(stats_half_t), (K, K))
            models_former[trial, t] = m_former_estimated
            models_latter[trial, t] = m_latter_estimated
            
            stat = 0.5 / h *(scores[trial, t, m_estimated] + codelens[m_estimated] - stats_half_t[m_former_estimated, m_latter_estimated])
            stats_complete[trial, t] = stat
            
            if (m_estimated == m_latter_estimated):
                # no model change happened
                model_t = m_estimated
            else:
                model_t = m_latter_estimated
            
            if t >= 1:
                model_prev = models_estimated[trial, t-1]
                if model_t != model_prev:
                    n_change += 1            
            models_estimated[trial, t] = model_t
    
    return stats_complete, models_estimated, models_former, models_latter

In [14]:
def calc_stats_with_modelidx(scores, scores_f, scores_l, idxes_model, idxes_model_f, idxes_model_l, h):
    scores = np.array(scores)
    scores_f = np.array(scores_f)
    scores_l = np.array(scores_l)
    
    stats_complete = np.nan * np.ones((idxes_model.shape[0], idxes_model.shape[1]), dtype=np.float)
    for trial in range(idxes_model.shape[0]):
        for t in range(h, idxes_model.shape[1]-h):
            stat = 0.5/h * (scores[trial, t, int(idxes_model[trial, t])] - \
                            (scores_f[trial, t, int(idxes_model_f[trial, t])] + \
                            scores_l[trial, t, int(idxes_model_l[trial, t])] ))
            stats_complete[trial, t] = stat
            
    return stats_complete

In [15]:
def codelen_integer(k):
    codelen = np.log(2.865)
    k = np.log(k)
    while k >= 0.0:
        codelen += k
        k = np.log(k)
        
    return codelen

In [16]:
with open(os.path.join(indir, 'X_gradual.pkl'), 'rb') as f:
    X_all = pickle.load(f)
with open(os.path.join(indir, 'Z_gradual.pkl'), 'rb') as f:
    Z_true_all = pickle.load(f)

In [17]:
X_all.shape

(8, 90, 100, 100)

In [18]:
def estimate_sbm_each_trial(X, trial, K, T):
    pi_list_trial = []
    theta_list_trial = []
    z_list_trial = []
    
    for t in tqdm.tqdm(range(T)):
        seed = trial * T + t
        numpy2ri.activate()
        sbm = blockmodels.BM_bernoulli(membership_type="SBM", 
                                       adj=np.array(X[trial, t, :, :]),
                                       verbosity=0,
                                       exploration_factor=1.5,
                                       explore_min=K,
                                       explore_max=K)

        estimate = dollar(sbm, "estimate")
        estimate()

        pi_list = []
        theta_list = []
        z_posterior_list = []
        for k in range(K):
            n_clusters = k + 1
            theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "pi"))
            z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
            pi = np.sum(z_posterior, axis=0) + 10 * EPS
            pi /= np.sum(pi)

            theta_list.append(theta)
            z_posterior_list.append(z_posterior)
            pi_list.append(pi)

        numpy2ri.deactivate()
        
        pi_list_trial.append(pi_list)
        theta_list_trial.append(theta_list)
        z_list_trial.append(z_posterior_list)

    return pi_list_trial, theta_list_trial, z_list_trial

In [19]:
EPS = np.finfo(np.float).eps

N_trial = X_all.shape[0]
T = X_all.shape[1]

K = 10

pi1 = None
pi2 = None
a0 = 1.0
b0 = 1.0
ratio = 0.02

pi_all = []
theta_all = []
z_all = []

for trial in range(N_trial):
    pi1 = None
    pi2 = None
    theta = None
    
    pi_list_trial = []
    theta_list_trial = []
    z_list_trial = []
    
    numpy2ri.activate()    
    
    for t in tqdm.tqdm(range(T)):
        seed = trial*T + t
        
        X = X_all[trial, t, :, :]
            
        sbm = blockmodels.BM_bernoulli(membership_type="SBM", adj=np.array(X),
                                           verbosity=0,
                                           exploration_factor=1.5,
                                           explore_min=K,
                                           explore_max=K)

        estimate = dollar(sbm, "estimate")
        estimate()

        theta_list = []
        pi_list = []
        z_posterior_list = []
        for k in range(K):
            n_clusters = k + 1
            theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "pi"))
            z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
            pi = np.sum(z_posterior, axis=0) + 10 * EPS
            pi /= np.sum(pi)

            theta_list.append(theta)
            z_posterior_list.append(z_posterior)
            pi_list.append(pi)
        
        pi_list_trial.append(pi_list)
        theta_list_trial.append(theta_list)
        z_list_trial.append(z_posterior_list)
        
    numpy2ri.deactivate()

    pi_all.append(pi_list_trial)
    theta_all.append(theta_list_trial)
    z_all.append(z_list_trial)

    with open(os.path.join(outdir, 'pi_gradual.pkl'), 'wb') as f:
        pickle.dump(pi_all, f)
    with open(os.path.join(outdir, 'theta_gradual.pkl'), 'wb') as f:
        pickle.dump(theta_all, f)
    with open(os.path.join(outdir, 'z_gradual.pkl'), 'wb') as f:
        pickle.dump(z_all, f)

  0%|          | 0/90 [00:00<?, ?it/s]

  1%|          | 1/90 [00:10<16:14, 10.94s/it]

  2%|▏         | 2/90 [00:19<15:12, 10.37s/it]

  3%|▎         | 3/90 [00:28<14:22,  9.91s/it]

  4%|▍         | 4/90 [00:36<13:02,  9.10s/it]

...

 96%|█████████▌| 86/90 [09:21<00:21,  5.26s/it]

 97%|█████████▋| 87/90 [09:30<00:18,  6.17s/it]

 98%|█████████▊| 88/90 [09:35<00:11,  5.86s/it]

 99%|█████████▉| 89/90 [09:43<00:06,  6.59s/it]

100%|██████████| 90/90 [09:50<00:00,  6.57s/it]


In [20]:
with open(os.path.join(outdir, 'pi_gradual.pkl'), 'rb') as f:
    pi_all = pickle.load(f)
with open(os.path.join(outdir, 'theta_gradual.pkl'), 'rb') as f:
    theta_all = pickle.load(f)
with open(os.path.join(outdir, 'z_gradual.pkl'), 'rb') as f:
    Z_all = pickle.load(f)

In [21]:
X_all.shape

(8, 90, 100, 100)

In [22]:
# relabeling
N_trial = X_all.shape[0]
K = 10
T = X_all.shape[1]

numpy2ri.activate()
for trial in tqdm.tqdm(range(N_trial)):
    for k in range(1, K):
        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(10)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(10)])
        )        
        permutations = np.array(dollar(run, "permutations"))
        for t in range(10):
            Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[t, :]-1]

        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(9, 14)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(9, 14)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(9, 14)):
            if t == 9:
                for tt in range(10):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]
        
        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(13, 35)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(13, 35)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(13, 35)):
            if t == 13:
                for tt in range(14):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]

        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(34, 40)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(34, 40)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(34, 40)):
            if t == 34:
                for tt in range(35):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]

        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(39, 59)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(39, 59)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(39, 59)):
            if t == 39:
                for tt in range(40):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]

        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(58, 69)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(58, 69)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(58, 69)):
            if t == 58:
                for tt in range(59):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]

        run = label_switching.ecr_iterative_2(
            z=np.vstack([
                  np.argmax(Z_all[trial][t][k], axis=1).reshape(1, -1) for t in range(68, 90)
            ]) + 1,
            K=k+1,
            p=np.stack([Z_all[trial][t][k] for t in range(68, 90)])
        )
        permutations = np.array(dollar(run, "permutations"))
        for i, t in enumerate(range(68, 90)):
            if t == 68:
                for tt in range(69):
                    Z_all[trial][tt][k] = Z_all[trial][tt][k][:, permutations[i, :]-1]
            else:
                Z_all[trial][t][k] = Z_all[trial][t][k][:, permutations[i, :]-1]
            
numpy2ri.deactivate()

100%|██████████| 8/8 [00:24<00:00,  3.10s/it]


In [23]:
with open(os.path.join(outdir, 'z_gradual.pkl'), 'wb') as f:
    pickle.dump(Z_all, f)

In [24]:
n_trial = X_all.shape[0]
T = X_all.shape[1]
K = 10

for h in [1, 2, 3]:
    dnml_whole_list, nml_x_z_whole_list, nml_z_whole_list = \
        np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K))
    dnml_former_list, nml_x_z_former_list, nml_z_former_list = \
        np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K))
    dnml_latter_list, nml_x_z_latter_list, nml_z_latter_list = \
        np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K)), np.nan * np.ones((n_trial, T, K))

    for trial in tqdm.tqdm(range(n_trial)):
        for t in tqdm.tqdm(range(h, T-h)):
            for k in range(K):
                t_start = t-h
                t_end = t+h

                # whole
                n_whole = np.sum([[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == v) for v in range(k+1)] 
                                  for tt in range(t_start, t_end)], axis=0)

                theta_hat_whole = np.sum(
                        [[[np.sum(X_all[trial][tt][np.argmax(Z_all[trial][tt][k], axis=1) == k1, :][:, np.argmax(Z_all[trial][tt][k], axis=1) == k2])
                           for k2 in range(k+1)] for k1 in range(k+1)] 
                           for tt in range(t_start, t_end)], axis=0) / \
                        np.sum(
                            [[[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k1) * 
                               np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k2) 
                            for k2 in range(k+1)] for k1 in range(k+1)] 
                            for tt in range(t_start, t_end)], axis=0)
                theta_hat_whole = theta_hat_whole[n_whole !=0, :][:, n_whole !=0]
                pi_hat_whole = n_whole[n_whole !=0] /np.sum(n_whole)

                n_cluster_whole = len(pi_hat_whole)
                res_w = np.array([calc_dnml_by_prob(X_all[trial, tt, :, :], 
                                                    Z_all[trial][tt][k], 
                                                    pi_hat_whole, theta_hat_whole,
                                                    n_cluster_whole, n_cluster_whole) 
                                  for tt in range(t_start, t_end)])

                dnml_whole = np.sum(res_w[:, 0])
                nml_x_z_whole = np.sum(res_w[:, 1])
                nml_z_whole = np.sum(res_w[:, 2])

                val = dnml_whole_list[trial, t, n_cluster_whole-1]
                if (np.isnan(val) | ( (np.isfinite(val)) & (dnml_whole < val) )):
                    dnml_whole_list[trial, t, n_cluster_whole-1] = dnml_whole
                    nml_x_z_whole_list[trial, t, n_cluster_whole-1] = nml_x_z_whole
                    nml_z_whole_list[trial, t, n_cluster_whole-1] = nml_z_whole

                # former
                n_former = np.sum([[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == v) for v in range(k+1)] 
                                   for tt in range(t_start, t_start+h)], axis=0)

                theta_hat_former = np.sum(
                        [[[np.sum(X_all[trial][tt][np.argmax(Z_all[trial][tt][k], axis=1) == k1, :][:, np.argmax(Z_all[trial][tt][k], axis=1) == k2])
                            for k2 in range(k+1)] for k1 in range(k+1)] 
                         for tt in range(t_start, t_start+h)], axis=0) / \
                        np.sum(
                            [[[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k1) * np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k2) 
                            for k2 in range(k+1)] for k1 in range(k+1)] 
                            for tt in range(t_start, t_start+h)], axis=0)

                theta_hat_former = theta_hat_former[n_former !=0, :][:, n_former !=0]
                pi_hat_former = n_former[n_former !=0] /np.sum(n_former)

                n_cluster_former = len(pi_hat_former)
                res_f = np.array([calc_dnml_by_prob(X_all[trial, tt, :, :], 
                                                    Z_all[trial][tt][k], 
                                                    pi_hat_former, theta_hat_former,
                                                    n_cluster_former, n_cluster_former) 
                                  for tt in range(t_start, t_start+h)])

                dnml_former = np.sum(res_f[:, 0])
                nml_x_z_former = np.sum(res_f[:, 1])
                nml_z_former = np.sum(res_f[:, 2])

                val = dnml_former_list[trial, t, n_cluster_former-1]
                if (np.isnan(val) | ( (np.isfinite(val)) & (dnml_former < val) )):
                    dnml_former_list[trial, t, n_cluster_former-1] = dnml_former
                    nml_x_z_former_list[trial, t, n_cluster_former-1] = nml_x_z_former
                    nml_z_former_list[trial, t, n_cluster_former-1] = nml_z_former

                # latter
                n_latter = np.sum([[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == v) for v in range(k+1)]
                                   for tt in range(t_start+h, t_end)], axis=0)

                theta_hat_latter = np.sum(
                         [[[np.sum(X_all[trial][tt][np.argmax(Z_all[trial][tt][k], axis=1) == k1, :][:, np.argmax(Z_all[trial][tt][k], axis=1) == k2])
                            for k2 in range(k+1)] for k1 in range(k+1)] for tt in range(t_start+h, t_end)], axis=0) / \
                        np.sum(
                         [[[np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k1) * 
                            np.sum(np.argmax(Z_all[trial][tt][k], axis=1) == k2) 
                            for k2 in range(k+1)] for k1 in range(k+1)] for tt in range(t_start+h, t_end)], axis=0)

                theta_hat_latter = theta_hat_latter[n_latter != 0, :][:, n_latter !=0]
                pi_hat_latter = n_latter[n_latter != 0] /np.sum(n_latter)

                n_cluster_latter = len(pi_hat_latter)
                res_l = np.array([calc_dnml_by_prob(X_all[trial, tt, :, :], 
                                                    Z_all[trial][tt][k], 
                                                    pi_hat_latter, theta_hat_latter,
                                                    n_cluster_latter, n_cluster_latter) 
                                  for tt in range(t_start+h, t_end)])

                dnml_latter = np.sum(res_l[:, 0])
                nml_x_z_latter = np.sum(res_l[:, 1])
                nml_z_latter = np.sum(res_l[:, 2])

                val = dnml_latter_list[trial, t, n_cluster_latter-1]
                if (np.isnan(val) | ( (np.isfinite(val)) & (dnml_latter < val) )):
                    dnml_latter_list[trial, t, n_cluster_latter-1] = dnml_latter
                    nml_x_z_latter_list[trial, t, n_cluster_latter-1] = nml_x_z_latter
                    nml_z_latter_list[trial, t, n_cluster_latter-1] = nml_z_latter

    with open(os.path.join(outdir, 'dnml_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(dnml_whole_list, f)
    with open(os.path.join(outdir, 'dnml_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(dnml_former_list, f)
    with open(os.path.join(outdir, 'dnml_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(dnml_latter_list, f)

    with open(os.path.join(outdir, 'nml_x_z_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_x_z_whole_list, f)
    with open(os.path.join(outdir, 'nml_x_z_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_x_z_former_list, f)
    with open(os.path.join(outdir, 'nml_x_z_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_x_z_latter_list, f)

    with open(os.path.join(outdir, 'nml_z_all_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_z_whole_list, f)
    with open(os.path.join(outdir, 'nml_z_f_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_z_former_list, f)
    with open(os.path.join(outdir, 'nml_z_l_h' + str(h) + '.pkl'), 'wb') as f:
        pickle.dump(nml_z_latter_list, f)

  8%|▊         | 7/88 [00:04<00:49,  1.63it/s]/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in true_divide

  9%|▉         | 8/88 [00:04<00:43,  1.85it/s]/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide

100%|██████████| 8/8 [10:48<00:00, 81.07s/it]
